In [40]:
import pandas as pd
import json

# Load the datasets
behaviors_df = pd.read_parquet('../data/eb_nerd/train/behaviors.parquet')
history_df = pd.read_parquet('../data/eb_nerd/train/history.parquet')
articles_df = pd.read_parquet('../data/eb_nerd/train/articles.parquet')

behaviors_df.head(10)

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0
1,152513,9778745.0,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, 9778623, 9089120, 9778661, ...",[9778661],150224,False,NaN,NaN,NaN,False,298,2.0,48.0
2,155390,NaN,2023-05-24 07:30:33,45.0,NaN,1,"[9778369, 9777856, 9778500, 9778021, 9778627, ...",[9777856],160892,False,NaN,NaN,NaN,False,401,215.0,100.0
3,214679,NaN,2023-05-23 05:25:40,33.0,NaN,2,"[9776715, 9776406, 9776566, 9776071, 9776808, ...",[9776566],1001055,False,NaN,NaN,NaN,False,1357,40.0,47.0
4,214681,NaN,2023-05-23 05:31:54,21.0,NaN,2,"[9775202, 9776855, 9776688, 9771995, 9776583, ...",[9776553],1001055,False,NaN,NaN,NaN,False,1358,5.0,49.0
5,214684,NaN,2023-05-23 05:32:21,10.0,NaN,2,"[9776508, 9767490, 9776049, 9776544, 9776551, ...",[9776508],1001055,False,NaN,NaN,NaN,False,1358,52.0,100.0
6,214691,NaN,2023-05-23 05:30:46,18.0,NaN,2,"[9759955, 9776449, 9775804, 9776369, 9488213, ...",[9776691],1001055,False,NaN,NaN,NaN,False,1358,4.0,37.0
7,369958,NaN,2023-05-24 14:25:56,16.0,NaN,2,"[9776023, 9778158, 9776929, 9527795, 7594265]",[9778158],1469458,False,NaN,NaN,NaN,False,1623,0.0,NaN
8,369959,NaN,2023-05-24 14:23:14,161.0,NaN,2,"[9779186, 9779289, 9777397, 9779185, 9778813, ...",[9779071],1469458,False,NaN,NaN,NaN,False,1623,16.0,NaN
9,370414,NaN,2023-05-24 14:48:54,9.0,NaN,2,"[9779408, 9779377, 9779289, 9778945, 9777182, ...",[9777182],1470585,False,NaN,NaN,NaN,False,1678,13.0,41.0


In [38]:
articles_df.head(10)

# Create a mapping from article_id to title
article_id_to_title = dict(zip(articles_df['article_id'].astype(str), articles_df['title']))

In [39]:
# Create train_infor_his
train_infor_his = {}
for _, row in history_df.iterrows():
    user_id = str(row['user_id'])
    clicked_articles = [article_id_to_title.get(str(article), str(article)) for article in row['article_id_fixed']]  # Replace with titles
    train_infor_his[user_id] = clicked_articles

# Save train_infor_his to a JSON file
with open('train_infor_his.json', 'w') as f:
    json.dump(train_infor_his, f)

# Create train_interaction
train_interaction = []
for _, row in behaviors_df.iterrows():
    user_id = str(row['user_id'])
    clicked_articles = row['article_ids_clicked']
    inview_articles = row['article_ids_inview']
    
    # Check if both clicked_articles and inview_articles are not empty
    if pd.notna(clicked_articles).any() and pd.notna(inview_articles).any():  
        clicked_articles = [article_id_to_title.get(str(article), str(article)) for article in clicked_articles if pd.notna(article)] 
        inview_articles = [article_id_to_title.get(str(article), str(article)) for article in inview_articles if pd.notna(article)]  
        for clicked in clicked_articles:
            for non_clicked in inview_articles:
                if non_clicked not in clicked_articles:
                    train_interaction.append((user_id, clicked, non_clicked))

# Save train_interaction to a file
with open('train_interaction.txt', 'w') as f:
    for interaction in train_interaction:
        f.write(f"{interaction}\n")

# Print the first few entries to verify
print("train_infor_his sample:", list(train_infor_his.items())[:1])
print("train_interaction sample:", train_interaction[:5])

train_infor_his sample: [('13538', ['Coop nedlægger 180 stillinger efter historisk underskud', 'Flere hundrede danske fodboldbaner skal laves om', 'Coop nedlægger 180 stillinger efter historisk underskud', 'Giga-handel går i vasken', 'Coop nedlægger 180 stillinger efter historisk underskud', 'Kaos i Brøndby: Forhandlinger brudt sammen', 'Coop nedlægger 180 stillinger efter historisk underskud', 'Ikonisk café drejer nøglen om', 'Coop nedlægger 180 stillinger efter historisk underskud', 'Efter brat afsked: Mister 700.000 seere', 'Coop nedlægger 180 stillinger efter historisk underskud', 'Grisetransport væltet og brudt i brand', 'Dansktalende kvinde i syrisk fangelejr: - Det er ren overlevelse', 'Sigtelse: Mand holdt ung kvinde fanget og slog hende', 'Konkurs: Umådeligt ked af det', 'Gældsstyrelsen udbetaler godtgørelse: 57.750 kroner pr. person', 'Fire personer tiltalt i Frank-sagen', 'Tv-vært Jerry Springer er død', 'Ærlig McD: Færre fritter', 'EU strammer grebet: Disse 19 giganter bliv